In [34]:
import os 

In [35]:
pwd

'c:\\Users\\Admin\\Desktop\\text-summarizer\\text-summarizer-project'

In [36]:
os.chdir('c:/Users/Admin/Desktop/text-summarizer/text-summarizer-project/')

In [37]:
pwd

'c:\\Users\\Admin\\Desktop\\text-summarizer\\text-summarizer-project'

In [38]:
import sys
sys.path.append(r"c:\Users\Admin\Desktop\text-summarizer\text-summarizer-project\src")


In [39]:
from dataclasses import dataclass
from pathlib import Path
from dataclasses import dataclass
from pathlib import Path
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    learning_rate: float
    weight_decay: float
    warmup_steps: int
    logging_dir: Path
    logging_steps: int
    evaluation_strategy: str
    save_strategy: str
    save_total_limit: int
    fp16: bool
    push_to_hub: bool
    hub_model_id: str
    hub_token: str
        # <-- ADD 

In [40]:
from text_summarizer.utils.common import read_yaml, create_directories
from text_summarizer.constants import *

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        model_trainer_config = self.config.model_trainer

        model_trainer_dir = os.path.join(
            self.config.artifacts_root,
            model_trainer_config.root_dir
        )

        create_directories([model_trainer_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=model_trainer_dir,
            data_path=model_trainer_config.data_path,
            model_ckpt=model_trainer_config.model_ckpt,
            num_train_epochs=self.params.num_train_epochs,
            per_device_train_batch_size=self.params.per_device_train_batch_size,
            per_device_eval_batch_size=self.params.per_device_eval_batch_size,
            learning_rate=self.params.learning_rate,
            weight_decay=self.params.weight_decay,
            warmup_steps=self.params.warmup_steps,
            logging_dir=os.path.join(model_trainer_dir, model_trainer_config.logging_dir),
            logging_steps=model_trainer_config.logging_steps,
            evaluation_strategy=model_trainer_config.evaluation_strategy,
            save_strategy=model_trainer_config.save_strategy,
            save_total_limit=model_trainer_config.save_total_limit,
            fp16=model_trainer_config.fp16,
            push_to_hub=model_trainer_config.push_to_hub,
            hub_model_id=model_trainer_config.hub_model_id,
            hub_token=model_trainer_config.hub_token
        )

        return model_trainer_config
    

In [42]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk, load_dataset
import torch



In [43]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import os
import torch

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def train(self):
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)

        # Load tokenized dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        training_args = TrainingArguments(
    output_dir=self.config.root_dir,
    num_train_epochs=self.config.num_train_epochs,
    per_device_train_batch_size=self.config.per_device_train_batch_size,
    per_device_eval_batch_size=self.config.per_device_eval_batch_size,
    learning_rate=self.config.learning_rate,
    weight_decay=self.config.weight_decay,
    warmup_steps=self.config.warmup_steps,
    logging_dir=self.config.logging_dir,
    logging_steps=self.config.logging_steps,
    # Remove unsupported arguments:
    # evaluation_strategy=self.config.evaluation_strategy,
    # save_strategy=self.config.save_strategy,
    # save_total_limit=self.config.save_total_limit,
    # fp16=self.config.fp16,
    # push_to_hub=self.config.push_to_hub,
    # hub_model_id=self.config.hub_model_id,
    # hub_token=self.config.hub_token
)

        trainer = Trainer(
            model=model,
            args=training_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt['test'],
            eval_dataset=dataset_samsum_pt['validation']
        )

        trainer.train()

        # Save trained model and tokenizer
        model.save_pretrained(os.path.join(self.config.root_dir, 't5-summarizer'))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'tokenizer'))


In [44]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()

    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


2025-12-01 21:07:42,573 - INFO - common - YAML file: C:\Users\Admin\Desktop\text-summarizer\text-summarizer-project\config\config.yaml loaded successfully.
2025-12-01 21:07:42,590 - INFO - common - YAML file: C:\Users\Admin\Desktop\text-summarizer\text-summarizer-project\config\params.yaml loaded successfully.
2025-12-01 21:07:42,594 - INFO - common - Directory created at: artifacts
2025-12-01 21:07:42,599 - INFO - common - Directory created at: artifacts\model_trainer


FileNotFoundError: Directory data_transformation/samsum_dataset_t5 not found

'c:\\Users\\Admin\\Desktop\\text-summarizer\\text-summarizer-project'

In [ ]:
from text_summarizer.utils.common import read_yaml
p = read_yaml("config/params.yaml")
p


2025-11-28 01:32:05,228 - ERROR - common - Error reading YAML file at config/params.yaml: [Errno 2] No such file or directory: 'config/params.yaml'


FileNotFoundError: [Errno 2] No such file or directory: 'config/params.yaml'

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from datasets import load_from_disk
from text_summarizer.logging import logger
from text_summarizer.utils.common import create_directories
import torch
import os

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path            # tokenized dataset dir
    model_ckpt: str
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    learning_rate: float
    weight_decay: float
    warmup_steps: int
    logging_dir: Path
    logging_steps: int
    evaluation_strategy: str
    save_strategy: str
    save_total_limit: int
    fp16: bool
    push_to_hub: bool
    hub_model_id: str
    hub_token: str

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        create_directories([self.config.root_dir, self.config.logging_dir])
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

    def train(self):
        logger.info("Loading model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(self.device)

        logger.info(f"Loading tokenized dataset from {self.config.data_path}")
        dataset = load_from_disk(str(self.config.data_path))

        collator = DataCollatorForSeq2Seq(tokenizer, model=model)

        training_args = TrainingArguments(
            output_dir=str(self.config.root_dir),
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            learning_rate=self.config.learning_rate,
            weight_decay=self.config.weight_decay,
            warmup_steps=self.config.warmup_steps,
            logging_dir=str(self.config.logging_dir),
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            save_strategy=self.config.save_strategy,
            save_total_limit=self.config.save_total_limit,
            fp16=self.config.fp16,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            tokenizer=tokenizer,
            data_collator=collator
        )

        logger.info("Starting training...")
        trainer.train()

        out_dir = Path(self.config.root_dir) / "t5-summarizer"
        model.save_pretrained(str(out_dir))
        tokenizer.save_pretrained(str(out_dir))
        logger.info(f"Training complete. Model saved to: {out_dir}")